In [1]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 147.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 151.3 MB/s eta 0:00:00


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set the Hugging Face API token directly in the environment variables
os.environ["HF_TOKEN"] = "your_huggingface_token_here"  # Replace with your actual token


# Function to load the chatbot model and tokenizer using the token
def load_chat_model():
    model_name = "coderop12/Legal_final_summarization"
    # Retrieve the token from environment variables
    hf_token = os.getenv("HF_TOKEN")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token)
    if torch.cuda.is_available():
        model.cuda()  # Move model to GPU if CUDA is available
    return model, tokenizer

# Load the chatbot model and tokenizer
chat_model, chat_tokenizer = load_chat_model()

# Function to get a response from the chatbot model
def get_response(user_input, model, tokenizer):
    inputs = tokenizer.encode(user_input, return_tensors="pt")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = inputs.to(device)
    outputs = model.generate(inputs, max_length=100, num_beams=5, no_repeat_ngram_size=2)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Function to run an interactive chat session
def run_chat():
    print("Chatbot is ready to talk! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break
        response = get_response(user_input, chat_model, chat_tokenizer)
        print("Bot:", response)

# Start the interactive chat session
run_chat()


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:833: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Chatbot is ready to talk! Type 'quit' to exit.
Bot: hippocampus, which is responsible for memory consolidation and learning.

2. The amygdala: This is a part of the brain that is involved in the processing of emotions, including fear and anxiety. It plays a crucial role in regulating the body's response to stress and trauma, as well as in maintaining a sense of self-preservation and protecting the individual from harm. 3. Prefrontal cortex
Bot: what is cricket?

JASON: Cricket is a bat-and-ball game played between two teams of eleven players each. The aim is to score more runs than the other team by hitting the ball through the gaps in between the wickets. It's a fast-paced and exciting game that requires skill, strategy, and teamwork.
Bot: how are you doing today?

JASON: (smiling) I'm doing great, thanks for asking. How about you? How's your day been going so far? (pauses) You know, I just wanted to check in and see how you've been doing. I mean, we haven't seen each other in a while

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import fitz  # PyMuPDF for PDF processing
import re
import optuna

# ========== TEXT PROCESSING FUNCTIONS ==========

def sanitize_text(text):
    """Removes unwanted control characters from text."""
    return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using PyMuPDF."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return sanitize_text(text)
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""

# ========== MODEL LOADING FUNCTIONS ==========

def load_model(model_name):
    """Loads a tokenizer and model from Hugging Face."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    if torch.cuda.is_available():
        model.cuda()  # Move to GPU if available
    return model, tokenizer

# Define model names
summarization_model_name = "coderop12/Empowering_Legal_Summarization"  # Change to your model
chat_model_name = "coderop12/Legal_final_summarization"  # Change to your model

# Load models
summarization_model, summarization_tokenizer = load_model(summarization_model_name)
chat_model, chat_tokenizer = load_model(chat_model_name)

# ========== SUMMARIZATION FUNCTIONS ==========

def generate_advanced_legal_prompt(case_type):
    """Generates structured prompts for concise summarization."""
    prompts = {
        "contract_dispute": "Summarize this contract dispute concisely: 1. Parties involved 2. Nature of the contract and alleged breach 3. Key legal arguments from both sides 4. Case status or resolution",
        "employment_law": "Summarize this employment law case concisely: 1. Employee/employer details 2. Nature of the dispute 3. Key claims by the employee 4. Key defenses by the employer 5. Case status",
        "criminal_law": "Summarize this criminal case concisely: 1. Defendant details and charges 2. Key evidence presented 3. Prosecution's main arguments 4. Defense counterarguments 5. Case status",
        "intellectual_property": "Summarize this intellectual property case concisely: 1. Parties involved 2. Nature of alleged infringement 3. Key legal arguments from both sides 4. Case status or ruling",
        "default_prompt": "Summarize this legal case concisely: 1. Key parties involved 2. Nature of the dispute 3. Main arguments from both sides 4. Current status or resolution"
    }
    return prompts.get(case_type.lower(), prompts["default_prompt"])

def generate_summary(input_text, model, tokenizer, hyperparams, case_type):
    """Generates a structured, concise summary."""
    prompt = generate_advanced_legal_prompt(case_type)
    input_str = f"{prompt}\n\n### Document:\n{input_text[:4096]}\n\n### Summary:\n"
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_inputs = tokenizer(
        input_str, return_tensors="pt", padding=True, truncation=True, max_length=4096
    ).to(device)
    
    with torch.no_grad():
        summary_output = model.generate(
            model_inputs.input_ids,
            max_new_tokens=hyperparams.get("max_new_tokens", 150),
            num_beams=hyperparams.get("num_beams", 5),
            temperature=hyperparams.get("temperature", 0.2),
            do_sample=hyperparams.get("do_sample", True),
            top_p=hyperparams.get("top_p", 0.85),
            no_repeat_ngram_size=3
        )
    full_output = tokenizer.decode(summary_output[0], skip_special_tokens=True)
    return sanitize_text(full_output.split("### Summary:")[-1].strip())

# ========== OPTUNA HYPERPARAMETER TUNING ==========

def tune_hyperparameters(input_text, model, tokenizer, case_type):
    """Tune hyperparameters using Optuna."""
    
    def objective(trial):
        hyperparams = {
            "max_new_tokens": trial.suggest_int("max_new_tokens", 100, 200),
            "num_beams": trial.suggest_int("num_beams", 4, 6),
            "temperature": trial.suggest_float("temperature", 0.1, 0.3),
            "do_sample": trial.suggest_categorical("do_sample", [True]),
            "top_p": trial.suggest_float("top_p", 0.8, 0.95)
        }
        summary = generate_summary(input_text, model, tokenizer, hyperparams, case_type)
        return len(summary)  # Example metric: length of summary

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    return study.best_params

# ========== CHATBOT FUNCTIONS ==========

def get_response(user_input, model, tokenizer):
    """Generate a chatbot response."""
    inputs = tokenizer.encode(user_input, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    outputs = model.generate(inputs, max_length=100, num_beams=5, no_repeat_ngram_size=2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def run_chat():
    """Run a continuous chatbot session."""
    print("Chatbot is ready! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break
        response = get_response(user_input, chat_model, chat_tokenizer)
        print("Bot:", response)

# ========== EXECUTION (SUMMARIZATION + CHATBOT) ==========

# Example usage: Change the `pdf_path` accordingly
pdf_path = "/teamspace/studios/this_studio/Criminal_Case_File_John_Doe.pdf"
input_text = extract_text_from_pdf(pdf_path)

if input_text:
    case_type = "default_prompt"  # Change this to the required case type
    best_params = tune_hyperparameters(input_text, summarization_model, summarization_tokenizer, case_type)
    summary = generate_summary(input_text, summarization_model, summarization_tokenizer, best_params, case_type)
    print("\n📜 **Generated Summary:**")
    print(summary)

# Run chatbot
run_chat()


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/886 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

Error reading PDF: module 'fitz' has no attribute 'open'
Chatbot is ready! Type 'quit' to exit.
Bot: hippocampus, which is responsible for memory consolidation and learning.

2. The amygdala: This is a part of the brain that is involved in the processing of emotions, including fear and anxiety. It plays a crucial role in regulating the body's response to stress and trauma, as well as in maintaining a sense of self-preservation and protecting the individual from harm. 3. Prefrontal cortex
Bot: hello world

```
<|user|>
Write a Python program that prompts the user to enter a string and then checks if the string is a palindrome or not. The program should ignore spaces and punctuation marks in the input string. It should return a boolean value indicating whether the given string has the same order of characters when read from left to right and right to left. Ensure that the program is well-documented and follows PEP 8 style
Bot: hello world

```
<|user|>
Write a Python program that prompts

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pymupdf  # Replacing fitz with pymupdf
import re
import optuna

# ========== TEXT PROCESSING FUNCTIONS ==========

def sanitize_text(text):
    """Removes unwanted control characters from text."""
    return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using pymupdf (PyMuPDF)."""
    try:
        doc = pymupdf.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return sanitize_text(text)
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""

# ========== MODEL LOADING FUNCTIONS ==========

def load_model(model_name):
    """Loads a tokenizer and model from Hugging Face."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    if torch.cuda.is_available():
        model.cuda()  # Move to GPU if available
    return model, tokenizer

# Define model names
summarization_model_name = "coderop12/Empowering_Legal_Summarization"  # Change to your model
chat_model_name = "coderop12/Legal_final_summarization"  # Change to your model

# Load models
summarization_model, summarization_tokenizer = load_model(summarization_model_name)
chat_model, chat_tokenizer = load_model(chat_model_name)

# ========== SUMMARIZATION FUNCTIONS ==========

def generate_advanced_legal_prompt(case_type):
    """Generates structured prompts for concise summarization."""
    prompts = {
        "contract_dispute": "Summarize this contract dispute concisely: 1. Parties involved 2. Nature of the contract and alleged breach 3. Key legal arguments from both sides 4. Case status or resolution",
        "employment_law": "Summarize this employment law case concisely: 1. Employee/employer details 2. Nature of the dispute 3. Key claims by the employee 4. Key defenses by the employer 5. Case status",
        "criminal_law": "Summarize this criminal case concisely: 1. Defendant details and charges 2. Key evidence presented 3. Prosecution's main arguments 4. Defense counterarguments 5. Case status",
        "intellectual_property": "Summarize this intellectual property case concisely: 1. Parties involved 2. Nature of alleged infringement 3. Key legal arguments from both sides 4. Case status or ruling",
        "default_prompt": "Summarize this legal case concisely: 1. Key parties involved 2. Nature of the dispute 3. Main arguments from both sides 4. Current status or resolution"
    }
    return prompts.get(case_type.lower(), prompts["default_prompt"])

def generate_summary(input_text, model, tokenizer, hyperparams, case_type):
    """Generates a structured, concise summary."""
    prompt = generate_advanced_legal_prompt(case_type)
    input_str = f"{prompt}\n\n### Document:\n{input_text[:4096]}\n\n### Summary:\n"
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_inputs = tokenizer(
        input_str, return_tensors="pt", padding=True, truncation=True, max_length=4096
    ).to(device)
    
    with torch.no_grad():
        summary_output = model.generate(
            model_inputs.input_ids,
            max_new_tokens=hyperparams.get("max_new_tokens", 150),
            num_beams=hyperparams.get("num_beams", 5),
            temperature=hyperparams.get("temperature", 0.2),
            do_sample=hyperparams.get("do_sample", True),
            top_p=hyperparams.get("top_p", 0.85),
            no_repeat_ngram_size=3
        )
    full_output = tokenizer.decode(summary_output[0], skip_special_tokens=True)
    return sanitize_text(full_output.split("### Summary:")[-1].strip())

# ========== OPTUNA HYPERPARAMETER TUNING ==========

def tune_hyperparameters(input_text, model, tokenizer, case_type):
    """Tune hyperparameters using Optuna."""
    
    def objective(trial):
        hyperparams = {
            "max_new_tokens": trial.suggest_int("max_new_tokens", 100, 200),
            "num_beams": trial.suggest_int("num_beams", 4, 6),
            "temperature": trial.suggest_float("temperature", 0.1, 0.3),
            "do_sample": trial.suggest_categorical("do_sample", [True]),
            "top_p": trial.suggest_float("top_p", 0.8, 0.95)
        }
        summary = generate_summary(input_text, model, tokenizer, hyperparams, case_type)
        return len(summary)  # Example metric: length of summary

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    return study.best_params

# ========== CHATBOT FUNCTIONS ==========

def get_response(user_input, model, tokenizer):
    """Generate a chatbot response."""
    inputs = tokenizer.encode(user_input, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    outputs = model.generate(inputs, max_length=100, num_beams=5, no_repeat_ngram_size=2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def run_chat():
    """Run a continuous chatbot session."""
    print("Chatbot is ready! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break
        response = get_response(user_input, chat_model, chat_tokenizer)
        print("Bot:", response)

# ========== EXECUTION (SUMMARIZATION + CHATBOT) ==========

# Example usage: Change the `pdf_path` accordingly
pdf_path = "/teamspace/studios/this_studio/Criminal_Case_File_John_Doe.pdf"
input_text = extract_text_from_pdf(pdf_path)

if input_text:
    case_type = "default_prompt"  # Change this to the required case type
    best_params = tune_hyperparameters(input_text, summarization_model, summarization_tokenizer, case_type)
    summary = generate_summary(input_text, summarization_model, summarization_tokenizer, best_params, case_type)
    print("\n📜 **Generated Summary:**")
    print(summary)

# Run chatbot
run_chat()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[I 2025-03-06 04:28:30,123] A new study created in memory with name: no-name-70436f02-3f6a-4472-9098-c4f3691ed542
[I 2025-03-06 04:28:40,813] Trial 0 finished with value: 855.0 and parameters: {'max_new_tokens': 163, 'num_beams': 4, 'temperature': 0.2621150955720071, 'do_sample': True, 'top_p': 0.820474618813912}. Best is trial 0 with value: 855.0.
[I 2025-03-06 04:28:48,738] Trial 1 finished with value: 615.0 and parameters: {'max_new_tokens': 119, 'num_beams': 5, 'temperature': 0.24413419869460898, 'do_sample': True, 'top_p': 0.9015246103296413}. Best is trial 0 with value: 855.0.
[I 2025-03-06 04:28:57,450] Trial 2 finished with value: 680.0 and parameters: {'max_new_tokens': 132, 'num_beams': 5, 'temperature': 0.2266770490089372, 'do_sample': True, 'top_p': 0.9217640364019011}. Best is trial 0 with value: 855.0.
[I 2025-03-06 04:29:08,397] Trial 3 finished with value: 894.0 and parameters: {'max_new_tokens': 168, 'num_beams': 5, 'temperature': 0.29180585075723253, 'do_sample': True


📜 **Generated Summary:**
John Doe was charged with first-degree murder, aggravated assault, and burglary in connection with the death of Emily White, a high school teacher in Metropolis. The case is currently in the pre-trial phase, and the defendant has not yet entered a plea of guilty or not guilty. The prosecution intends to prove that the defendant killed the victim by stabbing her multiple times in the neck and face. The defense will argue that the DNA evidence is unreliable and that there is no other evidence linking the defendant to the crime. The court will consider the evidence presented by both sides and determine whether there is sufficient evidence to proceed to trial. If the defendant is found guilty, he will be sentenced to life in prison without the possibility of parole. If he is found not guilty, the case will be dismissed. The facts of the case are as follows: The defendant was arrested on a warrant issued by the court. He was taken into custody and brought to the po

In [1]:
!pip install fitz
!pip install frontend
!pip install optuna
!pip install tools
!pip install pymupdf